# --- Day 16: Reindeer Maze ---
https://adventofcode.com/2024/day/16

In [1]:
def getMaze():
    with open("maze.txt") as file:
        return file.read()

In [23]:
from heapq import heappush, heappop

# Define a Node class where information about a current location (coordinate) is held
class Node:
    def __init__(self, coords: tuple[int], parent, direction: str, cost: int=-1):
        self.coords = coords # Coordinates
        self.parent = parent # This is the parent node that this node came from
        self.direction = direction
        self.cost = cost

    # Returns true if state (coordinates) are equal
    def __eq__(self, other):
        if type(self) == type(other):
            return self.coords == other.coords
        return self.coords == other
    
    def __lt__(self, other):
        return self.cost < other.cost
    
    def __repr__(self):
        return f"{self.coords}, {self.direction}, {self.cost}"

# Formatting
maze = getMaze()
maze = maze.split("\n")
rows, cols = len(maze), len(maze[0])

# Find the start and the end node
for i in range(rows):
    for j in range(cols):
        if maze[i][j] == "S":
            start = (i,j)
        if maze[i][j] == "E":
            end = (i,j)

# Search through the maze until we hit the end
def dijkstra(maze, start, end):
    startNode = Node(start, None, "right", 0) # Create a start node with coords

    # Create frontier
    frontier = []
    heappush(frontier, startNode)

    # Create explored 
    explored = dict()
    explored[startNode.coords] = startNode.cost

    # Loop until frontier is empty
    while frontier:
        currentNode = heappop(frontier)

        # If we've made it to the end, then we can return the current node
        if currentNode == end:
            return currentNode
        
        # Check all successors
        currentCoords = currentNode.coords
        newCost = currentNode.cost + 1
        up = Node((currentCoords[0] - 1, currentCoords[1]), parent=currentNode, direction="up", cost=newCost)
        down = Node((currentCoords[0] + 1, currentCoords[1]), parent=currentNode, direction="down", cost=newCost)
        left = Node((currentCoords[0], currentCoords[1] - 1), parent=currentNode, direction="left", cost=newCost)
        right = Node((currentCoords[0], currentCoords[1] + 1), parent=currentNode, direction="right", cost=newCost)

        # Loop through each child
        for child in [up, down, left, right]:
            # If the spot is open or we make it to the end
            if maze[child.coords[0]][child.coords[1]] in [".", "E"]:

                # If there is a direction change, add 1000 to the cost
                if currentNode.direction != child.direction:
                    child.cost += 1000

                # If new coordinate or cheaper coordinate, add to frontier and explored
                if child.coords not in explored.keys() or explored[child.coords] > child.cost:
                    explored[child.coords] = child.cost
                    heappush(frontier, child)

    # If we made it to the end, then we did not find our goal :(
    return None

print(f"Lowest possible reindeer score: {dijkstra(maze, start, end).cost}")

Lowest possible reindeer score: 98416


# --- Part Two ---

In [4]:
from heapq import heappush, heappop
import sys
sys.setrecursionlimit(15000) # Change recursion limit

def printMaze(maze, coordsVisited: list[tuple[int]]=[]):
    for i in range(len(maze)):
        for j in range(len(maze[i])):
            if (i,j) in coordsVisited:
                print("O", end="")
            else:
                print(maze[i][j], end="")
        print()
    print()

# Define a Node class where information about a current location (coordinate) is held
class Node:
    def __init__(self, coords: tuple[int], parent, direction: str, cost: int):
        self.coords = coords # Coordinates
        self.parent = parent # This is the node that this node came from
        self.direction = direction
        self.cost = cost

    # Returns true if state (coordinates) are equal
    def __eq__(self, other):
        if type(self) == type(other):
            return self.coords == other.coords
        return self.coords == other
    
    def __lt__(self, other):
        return self.cost < other.cost
    
    # Return all coordinates that have been visited before this node
    @property
    def coordsVisited(self) -> list[tuple[int]]:
        currentNode = self
        coords = []

        # Loop through until we get to the first node
        while currentNode.parent != None:
            coords.append(currentNode.coords)
            currentNode = currentNode.parent
        # Add final coordinate to the list of coords
        coords.append(currentNode.coords)
        return coords
    
    def __repr__(self):
        return f"({self.coords} | {self.direction} | {self.cost})"

# Formatting
maze = getMaze()
maze = maze.split("\n")
rows, cols = len(maze), len(maze[0])

# Find the start and the end node
for i in range(rows):
    for j in range(cols):
        if maze[i][j] == "S":
            start = (i,j)
        if maze[i][j] == "E":
            end = (i,j)

# Search through the maze until we hit the end
def dijkstra(maze, start, end):
    startNode = Node(start, None, "right", 0) # Create a start node with coords

    # Create frontier
    frontier = []
    heappush(frontier, startNode)

    # Create explored 
    explored = dict()
    explored[startNode.coords] = startNode.cost

    # Loop until frontier is empty
    while frontier:
        currentNode = heappop(frontier)

        # If we've made it to the end, then we can return the current node
        if currentNode == end:
            return currentNode
        
        # Check all successors
        currentCoords = currentNode.coords
        newCost = currentNode.cost + 1
        up = Node((currentCoords[0] - 1, currentCoords[1]), parent=currentNode, direction="up", cost=newCost)
        down = Node((currentCoords[0] + 1, currentCoords[1]), parent=currentNode, direction="down", cost=newCost)
        left = Node((currentCoords[0], currentCoords[1] - 1), parent=currentNode, direction="left", cost=newCost)
        right = Node((currentCoords[0], currentCoords[1] + 1), parent=currentNode, direction="right", cost=newCost)

        # Loop through each child
        for child in [up, down, left, right]:
            # If the spot is open or we make it to the end
            if maze[child.coords[0]][child.coords[1]] in [".", "E"]:

                # If there is a direction change, add 1000 to the cost
                if currentNode.direction != child.direction:
                    child.cost += 1000

                # If new coordinate or cheaper coordinate, add to frontier and explored
                if child.coords not in explored.keys() or explored[child.coords] > child.cost:
                    explored[child.coords] = child.cost
                    heappush(frontier, child)

    # If we made it to the end, then we did not find our goal :(
    return None

def getAllBestPaths(maze, start, end, minCost, currentNode=None):

    # This should keep track of all end nodes
    allPaths = []
    
    # Base case (if we've made it to the end)
    if currentNode.coords == end:
        return [currentNode]
    # If we're already past the minCost, don't bother continuing down that path
    if currentNode.cost > minCost:
        return []
    # If we're on a node we've already visited, then this is not a best path
    if currentNode.parent and currentNode.coords in currentNode.parent.coordsVisited:
        return []
    
    # Get all possible next nodes
    currentCoords = currentNode.coords
    newCost = currentNode.cost + 1
    up = Node((currentCoords[0] - 1, currentCoords[1]), parent=currentNode, direction="up", cost=newCost)
    down = Node((currentCoords[0] + 1, currentCoords[1]), parent=currentNode, direction="down", cost=newCost)
    left = Node((currentCoords[0], currentCoords[1] - 1), parent=currentNode, direction="left", cost=newCost)
    right = Node((currentCoords[0], currentCoords[1] + 1), parent=currentNode, direction="right", cost=newCost)

    for child in [up, down, left, right]:
        # If the child is an open space that does not go backwards, go down that path
        if maze[child.coords[0]][child.coords[1]] in [".", "E"] and child != currentNode.parent:

            #print(f"This works: {child.coords}")
            # If there is a direction change, add 1000 to the cost
            if currentNode.direction != child.direction:
                child.cost += 1000

            allPaths.extend(getAllBestPaths(maze, start, end, minCost, currentNode=child))

    # Return all paths
    return allPaths

# Get the min cost (this is what we will filter)
minCost: int = dijkstra(maze, start, end).cost

# List of nodes
allBestPaths = getAllBestPaths(maze, start, end, minCost, Node(start, None, "right", 0))

# Get the number of best tiles
bestTiles = []
for path in allBestPaths:
    bestTiles.extend(path.coordsVisited)

print(f"Number of tiles part of at least one best path: {len(set(bestTiles))}")

Number of tiles part of at least one best path: 64


In [3]:
# Ideas so far - 
# Use dijkstra's algorithm to find shortest possible path
# Find all paths of that length